In [1]:
import re
import numpy as np
import pandas as pd
import requests,time
import datetime
from scipy import stats
import csv
from bs4 import BeautifulSoup
from requests_html import HTMLSession
import json
import webbrowser
import warnings
import PyPDF2
import io
import requests
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS


In [2]:
ls16_sess = [(pd.read_json(f'16session{i}.json', convert_dates=False)) for i in range(2,18)]
ls17_sess = [(pd.read_json(f'17session{i}.json', convert_dates=False)) for i in range(1,13)]

ls_sess = [*ls16_sess ,*ls17_sess]

# * appending all the sessions to the one dataframe
concatdf = pd.concat(ls_sess, ignore_index=True) # ? only 6 sessions rn

# ? cleaning the data for member names and dates 
# *  Date formatting 
concatdf['date'] = concatdf['date'].str.replace('.','')
concatdf['date'] = (concatdf['date'].str[-4:]+ concatdf['date'].str[2:4] + concatdf['date'].str[:2]).apply(int)  # ? date formating
# *  Member name formatting
concatdf['memberstring'] = [','.join(map(str, l)) for l in concatdf['member']] # ? cleaning the data for member
fourth_column = concatdf.pop('memberstring')
concatdf.insert(4,'memberstring',fourth_column)
concatdf["memberstring"].str.strip()

concatdf["type"].str.strip()
# * _________________________________________________________________________



# concatdf.sort_values(by = 'date')


# * total dates, subjects unique ________________________________________________________________
subjects_unique = concatdf['subjects']
subjects_unique.unique()
print(len(subjects_unique.unique()))
print(len(subjects_unique))
session_dates = concatdf['date']
len(session_dates.unique())

#concatdf
# ? Choose a range of dates 
start_date = 20140707
end_date =  20160303
concatdf['questionText'] = concatdf.questionText.str.replace("\n\r\n","")
#concatdf = concatdf.query(f'date >= {start_date}  & date <= {end_date}') # 20140707 to 20230811
warnings.filterwarnings("ignore",'This pattern is interpreted as a regular expression, and has match groups.To actually get the groups, use str.extract.')
concatdf = concatdf.sort_values(by = 'date', ignore_index= True)

loks16_mem = pd.read_excel("LS16MEMSTATE.xlsx")
loks17_mem = pd.read_excel("LS17MEMSTATE.xlsx")

108948
134866


In [3]:

def ls_frame(start_date, end_date):
   """_returns the datframe with question for the concerned time frame_

    Args:
        start_date (_int_): _the starting date >= 20140707 for now_
        end_date (_type_): _the last date <= 20230811_

    Returns:
        _dataframe_: _with date constraints_
    """   ''''''
   ls_frame = concatdf.query(f'date >= {start_date}  & date <= {end_date}') # 20140707 to 20230811
   return ls_frame


def pdf_indexes(dataframe):
    '''returns indexes of dataframe for column questionsFilePath, links which are pdf files'''
    
    alltr_indexes = []
    allfa_indexes = []

    for i in range(19097,dataframe.index[-1]):          #for word cloud
        val = dataframe.questionsFilePath[i]
        if 'pdf' in val:
            alltr_indexes.append(i)
        else:
            allfa_indexes.append(i)

    return alltr_indexes, allfa_indexes




def subjects_uniqueliststr(dataframe):
    '''Makes all the subjects as string
        for a dataframe, returns a string of all the subjects'''
    subjects_unique = dataframe['ministry']
    subjects_unique_list =  list(subjects_unique)
    subjects_unique_liststr = str(subjects_unique_list)   #for word cloud
    subjects_unique_liststr
    return subjects_unique_liststr




def scrape_pdf_and_questions(dataframe):
  '''Can scrape questions from pdf links in rows after regex confi and paste it on qtext columns
  given a dataframe'''

   #if "pdf" in url:
  indices = pdf_indexes(dataframe)[0]
  for ind in indices:                               # ? could just making a list of strings and pasting them sideways be faster
    url = dataframe.questionsFilePath[ind]
    if "https://" in url:
      r = requests.get(url)
      f = io.BytesIO(r.content)
      reader = PyPDF2.PdfReader(f)
      contents = reader.pages[0].extract_text().split('\n')
      string_contents = str(contents)
      dataframe.loc[ind,'questionText'] = string_contents
      #have to add regular_expressions
    
      #listn = [','.join(map(str, l)) for l in contents]



def total_club_allmem(dataframe):
    """_returns datframe containing questions that were clubbed_

    Args:
        dataframe (_type_): _description_

    Returns:
        _type_: _description_
    """    
    
    names = dataframe.loc[:,["member","memberstring"]]#.reset_index()
    #names = names.drop(["index"], axis = 1)
    clubbed_questions_df = names[names.loc[:,"member"].apply(len) != 1 ]
    return clubbed_questions_df






def  members_who_asked_quest(dataframe):
   ''' returns a list of members who asked questions either independently
   or clubbed, and total number of members who asked questions'''



   lok_sabha_members = [dataframe.loc[i,'member'][0] for i in dataframe.index]
   for i in total_club_allmem(dataframe).index:
    for j in range(len(dataframe.loc[i,'member'])):
        lok_sabha_members.append(dataframe.loc[i,'member'][j])
    
    lok_sabha_members = (set(lok_sabha_members))
    length = len(lok_sabha_members)
    return lok_sabha_members, length
   






def ministry_frequency(dataframe):
    '''returns a dataframe for a given dataframe highlighting, ministries and the amount of questions asked to them with
    respective proportion'''

    ministry_unique = dataframe['ministry']
    ministry_unique = pd.DataFrame(dataframe['ministry'].value_counts())
    ministry_unique['proportion%'] = ((dataframe['ministry'].value_counts())/dataframe['ministry'].shape[0])*100
    ministry_unique.loc['Column_Total']= ministry_unique.sum(numeric_only=True, axis=0)
    ministry_unique.reset_index(inplace=True)
    return ministry_unique





def clubbed_questions_of_a_member(member_name,dataframe):
    """_returns a dataframe containing all the questions of particular member that are clubbed with someone else_

    Args:
        member_name (_string_): _name of the member_
        dataframe (_dataframe_): _the parent dataframe from which you want to see_

    Returns:
        _dataframe_: _Questions that are clubbed of the member_
    """    
    member = dataframe.loc[dataframe["memberstring"].str.contains(r""+member_name+"", case=False, regex = False)]
    names = member.loc[:,:].reset_index()
    names = names.drop(["index"], axis = 1)
    clubbed_questions_df = names[names.loc[:,"memberstring"] != member_name]
    return clubbed_questions_df



def Questions_Clubbed_with_whom(member,dataframe):
    """_with whom and  the number of times questions got clubbed with_

    Args:
        member (_string_): _member's name acoording to the way in this data_
        dataframe (_dataframe_): _parent dataframe_

    Returns:
        _dataframe_: _containing with all the other members and number of times the member's questions were clubbed with_
    """
    membersask = members_who_asked_quest(dataframe)[0]
    member = clubbed_questions_of_a_member(member,dataframe)
    #supriya = supriya[supriya.memberstring.str.contains("DHANANJAY BHIMRAO MAHADIK", case = False)].shape[0]
   
    club_members = []
    for membs in sorted(membersask):
        count = member[member.memberstring.str.contains(r""+membs+"", case = False, regex =False)].shape[0]
        club_mem = {f"MemberName": membs,
                    "Questions_clubbed": count           
        }
        club_members.append(club_mem)
    club_members   = pd.DataFrame.from_dict(club_members)
    club_members = club_members.query("Questions_clubbed != 0")
    return club_members.sort_values(by = "Questions_clubbed", ascending = False)[:].reset_index(drop=True)


def get_star_unstar_tot_club(member_name,dataframe):
    """_memberstats_

    Args:
        member_name (_string_): _name of the member according to format CAPS_
        dataframe (_type_): _parent dataframe_

    Returns:
        _dictionary_: _of 6 keys and only one values for each_
    """    
    member_name = member_name
    warnings.filterwarnings("ignore", 'This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.')
    the_name = dataframe.loc[dataframe['memberstring'].str.contains(r""+member_name+"",case = False,regex = False)]
    clubbed_list = []

    club_count = clubbed_questions_of_a_member(member_name,dataframe).shape[0]


    unstarred = 0 if len(the_name.type.value_counts()) < 1 else the_name.type.value_counts().iloc[0]
    starred   = 0 if len(the_name.type.value_counts()) < 2 else the_name.type.value_counts().iloc[1] 
    dictionary_name_ques = {'MemberName' :f'{member_name}',
                            'Total_Questions' : unstarred+starred,
                            'Clubbed': club_count,
                            'Unstarred': unstarred,
                            'Starred' : starred,
                            '% Starred' : 1+starred/(1+unstarred+starred) -1

    }
    return dictionary_name_ques





def compdf_with_states(dataframe,lsno_member_list_states):
    '''comparison df goes into this, with cleaned member_list(second parameter) of states and party'''
    dataframe["State"] = "N/A"
    dataframe["Party Name"] = "N/A"
    for i in range(dataframe.shape[0]):
        ser = lsno_member_list_states.loc[:,"Name of Member"].str.contains(r""+dataframe.loc[i,"MemberName"]+"",case= False,regex= False,na=False)
        
        if len(ser[ser].index) != 0:
            dataframe.loc[i,"State"] = str(lsno_member_list_states.loc[ser[ser].index[0],"State"])
            dataframe.loc[i,"Party Name"] = str(lsno_member_list_states.loc[ser[ser].index[0],"Party Name"])
        
    return dataframe





list = ['a','b']
def comparison_df(dataframe,lsno_member_list_states):
    """_returns a dataframe for a given dataframe with members total_questions asked prop of 
   clubbed, starred and unstarred_

    Args:
        dataframe (_type_): _description_
        lsno_member_list_states (_type_): _description_

    Returns:
        _type_: _description_
    """
    data = []
    for member_name in members_who_asked_quest(dataframe)[0]:
        data.append(get_star_unstar_tot_club(member_name,dataframe))
        comparison_df = pd.DataFrame.from_dict(data).reset_index()
        comparison_df.MemberName = comparison_df.MemberName.replace(r'\s+', ' ', regex=True)
    return compdf_with_states(comparison_df.sort_values(by = "Starred", ascending = False, ignore_index= True),lsno_member_list_states)







def Type_of_questions_2ormore_membersasked(mem_list,dataframe):
    """_What subjects or ministry to questions were clubbed with a particular member_

    Args:
        mem_list (_list_): _list containg the primary filter name at the 0 index_
        dataframe (_dataframe_): _the parent dataframe_

    Returns:
        _dataframe_: _returning the instances of the members in the list asking clubbed questions or part of another_
    """     
    mem = dataframe.loc[dataframe['memberstring'].str.contains(r""+mem_list[0]+"", case= False,regex =False)]
    for i in range(1,len(mem_list)):
        mem = mem.loc[mem['memberstring'].str.contains(r""+mem_list[i]+"", case = False)].reset_index().drop("index", axis=1)
    return mem




def Na_states_Party(topstr,filename):
    """_creates and saves a csv of names that have N/A states and Party_

    Args:
        topstr (_dataframe_): _created using comparison df_
        filename (_string_): _name of the csv you want in the folder_
    """       
    stateNA = topstr[:][topstr.State == "N/A"]
    stateNA.loc[:,["MemberName","State","Party Name"]].to_csv(f"C:/Users/shubh/OneDrive/Desktop/Programs/Lok Sabha/{filename}.csv")



def statewise_comparison(comparison_df, prime_dataframe):
    """_statewise wise comparison of every state's total questionn, starred, unstarred and the number of members that 
    asked question from that state_

    Args:
        comparison_df (_type_): _description_
        prime_dataframe (_dataframe_): _description_

    Returns:
        _type_: _description_
    """
    statesinfolist =[]
    for  state_n in comparison_df.State.unique():  
        
        zi= comparison_df.groupby("State").get_group(state_n)



        lis = []
        for name in zi.MemberName:
            lis.append(prime_dataframe[prime_dataframe.memberstring.str.contains(name,case =False, regex = False)])
        state_mp = pd.concat(lis)
        state_mp.type.value_counts()
        state_dict = {'State' : state_n,
                    'Total_questions' : state_mp.shape[0],
                    'Unstarred': state_mp.type.value_counts().iloc[0],
                    'Starred': state_mp.type.value_counts().iloc[1] if len(state_mp.type.value_counts()) > 1 else 0 ,
                    'MembercountAsk': zi.shape[0],
                    
                    }
        statesinfolist.append(state_dict)
    statesinfolist = pd.DataFrame(statesinfolist).sort_values(by = 'State', ascending = True).reset_index(drop = True)

    return statesinfolist


def ministry_statecomp(topstr, prime_datframe):
    """_returns a dataframe containing every states number of times it asked questions to which ministry
    ministries are columns and count and proportion of questions addressed either starred or unstarred_

    Args:
        topstr (_dataframe_): _comparison dataframe_
        prime_datframe (_type_): _description_

    Returns:
        _type_: _description_
    """
    stateminfreq = []
    for state_n in topstr.State.unique():
        zi= topstr.groupby("State").get_group(state_n)



        lis = []
        for name in zi.MemberName:
            lis.append(prime_datframe[prime_datframe.memberstring.str.contains(name,case =False, regex = False)])
        state_mp = pd.concat(lis)


        minfreq = ministry_frequency(state_mp).transpose()
        minfreq.columns = minfreq.iloc[0]   
        minfreq = minfreq.iloc[1:]
        minfreq["State"] = state_n
        state_col = minfreq.pop("State")
        minfreq.insert(0,'State',state_col)
        stateminfreq.append(minfreq)
    minfreq = pd.concat(stateminfreq, axis= 0)
    return minfreq






What a member of a certain party clubs with another member of another party
    
1. So what type of questions, ministries  Supriya sule clubs with Eknath shinde?
2. Cleaning the warnings
3. Data plotting

In [4]:
ls_20142016 = ls_frame(20140707,20160303)

In [6]:
ls_20142016

,quesNo,subjects,lokNo,member,memberstring,ministry,type,date,questionText,answerText,questionsFilePath,questionsFilePathHindi,sessionNo,supplementaryQuestionResDtoList,supplementaryType
0,65,RAISING RETIREMENT AGE IN SICK CPSES,16,[FAIZAL P.P. MOHAMMED],FAIZAL P.P. MOHAMMED,HEAVY INDUSTRIES AND PUBLIC ENTERPRISES,UNSTARRED,20140707,(a) whether the Government proposes or has any...,\r\nTHE MINISTER OF STATE FOR HEAVY INDUSTRIES...,,NaN,2,NaN,False
1,63,MOBILE COMMUNICATION SYSTEM,16,[ARJUN RAM MEGHWAL],ARJUN RAM MEGHWAL,COMMUNICATIONS AND INFORMATION TECHNOLOGY,UNSTARRED,20140707,(a) whether the Government has taken note of ...,\r\n\n\nTHE MINISTER OF COMMUNICATIONS AND INF...,,NaN,2,NaN,False
2,62,DEMAND AND SUPPLY OF NATURAL GAS,16,[NALIN KUMAR KATEEL],NALIN KUMAR KATEEL,PETROLEUM AND NATURAL GAS,UNSTARRED,20140707,\r\n(a)\tthe domestic demand and supply of nat...,\r\n\n\nMINISTER OF STATE (I/C) IN THE MINISTR...,,NaN,2,NaN,False
3,61,REFORMS IN CRIMINAL LAW,16,[HANSRAJ GANGARAM AHIR],HANSRAJ GANGARAM AHIR,LAW AND JUSTICE,UNSTARRED,20140707,(a) whether the Government has asked the Law ...,\r\n\n\nMINISTER OF LAW AND JUSTICE & COMMUNIC...,,NaN,2,NaN,False
4,60,BUILDING FACILITIES FOR POST OFFICES,16,[RANGASWAMY DHRUVANARAYANA],RANGASWAMY DHRUVANARAYANA,COMMUNICATIONS AND INFORMATION TECHNOLOGY,UNSTARRED,20140707,(a) whether the Government has any proposal t...,\r\n\n\nTHE MINISTER OF COMMUNICATIONS AND INF...,,NaN,2,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29542,1233,Decline in Water Level of Wells,16,[PRATHAP SIMHA],PRATHAP SIMHA,"WATER RESOURCES, RIVER DEVELOPMENT AND GANAGA ...",UNSTARRED,20160303,NaN,NaN,https://sansad.in/getFile/loksabhaquestions/an...,https://sansad.in/getFile/loksabhaquestions/qh...,7,NaN,False
29543,1232,Inter-State River Water Sharing Disputes,16,"[BAHADUR SINGH KOLI, CH. MALLA REDDY, DUSHYANT...","BAHADUR SINGH KOLI,CH. MALLA REDDY,DUSHYANT SI...","WATER RESOURCES, RIVER DEVELOPMENT AND GANAGA ...",UNSTARRED,20160303,NaN,NaN,https://sansad.in/getFile/loksabhaquestions/an...,https://sansad.in/getFile/loksabhaquestions/qh...,7,NaN,False
29544,1231,PMSY for MGNREGA Workers,16,[VINOD KUMAR SONKAR],VINOD KUMAR SONKAR,RURAL DEVELOPMENT,UNSTARRED,20160303,NaN,NaN,https://sansad.in/getFile/loksabhaquestions/an...,https://sansad.in/getFile/loksabhaquestions/qh...,7,NaN,False
29545,1230,Research in Water Management,16,[KUNWAR PUSHPENDRA SINGH CHANDEL],KUNWAR PUSHPENDRA SINGH CHANDEL,"WATER RESOURCES, RIVER DEVELOPMENT AND GANAGA ...",UNSTARRED,20160303,NaN,NaN,https://sansad.in/getFile/loksabhaquestions/an...,https://sansad.in/getFile/loksabhaquestions/qh...,7,NaN,False


In [5]:
topstr = comparison_df(ls_20142016,loks16_mem)

In [98]:
topstr

,index,MemberName,Total_Questions,Clubbed,Unstarred,Starred,% Starred,State,Party Name
0,433,DHARMENDRA YADAV,428,411,386,42,0.097902,Uttar Pradesh,Samajwadi Party
1,131,SUPRIYA SADANAND SULE,483,454,444,39,0.080579,Maharashtra,Nationalist Congress Party
2,335,NISHIKANT DUBEY,339,217,303,36,0.105882,Jharkhand,Bharatiya Janata Party
3,155,RAJEEV SHANKARRAO SATAV,434,422,401,33,0.075862,Maharashtra,Indian National Congress
4,296,DHANANJAY BHIMRAO MAHADIK,465,462,433,32,0.068670,Maharashtra,Nationalist Congress Party
...,...,...,...,...,...,...,...,...,...
454,248,RAMESH CHANDAPPA JIGAJINAGI,1,0,1,0,0.000000,Karnataka,Bharatiya Janata Party
455,234,TAPAS MANDAL,2,0,2,0,0.000000,West Bengal,All India Trinamool Congress
456,224,BHAGAT SINGH KOSHYARI,10,7,10,0,0.000000,Uttarakhand,Bharatiya Janata Party
457,222,BIJOY CHANDRA BARMAN,1,0,1,0,0.000000,West Bengal,All India Trinamool Congress


In [108]:
statewise_comparison(topstr,ls_20142016)

,State,Total_questions,Unstarred,Starred,MembercountAsk
0,Andaman and Nicobar Islands,50,48,2,1
1,Andhra Pradesh,2292,2149,143,19
2,Arunachal Pradesh,130,123,7,1
3,Assam,769,726,43,12
4,Bihar,2575,2387,188,30
5,Chandigarh,106,97,9,1
6,Chhattisgarh,746,704,42,9
7,Dadra and Nagar Haveli,20,20,0,1
8,Dadra and Nagar Haveli and Daman and Diu,17,16,1,1
9,Gujarat,981,922,59,11


first ranking based on the unstarr starr proportion mps , then mean of the for all the state,  then this ratio to the no. of mps in the state


which state questions to which ministry more

Why are the amount of questions mismatching


because compa df takes into clubbed quesstions hance there is double counting for every club questions, but for the state_comparison all club questions are only one questons

In [166]:
ministry_statecomp(topstr, ls_20142016)



ministry,State,RAILWAYS,HUMAN RESOURCE DEVELOPMENT,HOME AFFAIRS,AGRICULTURE,FINANCE,HEALTH AND FAMILY WELFARE,COMMUNICATIONS AND INFORMATION TECHNOLOGY,"CONSUMER AFFAIRS, FOOD AND PUBLIC DISTRIBUTION","ENVIRONMENT, FORESTS AND CLIMATE CHANGE",...,STATISTICS AND PROGRAMME IMPLEMENTATION,STEEL,MINORITY AFFAIRS,OVERSEAS INDIAN AFFAIRS,CORPORATE AFFAIRS,DEVELOPMENT OF NORTH EASTERN REGION,"SKILL DEVELOPMENT, ENTREPRENEURSHIP,YOUTH AFFAIRS AND SPORTS",PARLIAMENTARY AFFAIRS,Column_Total,PRIME MINISTER
count,Uttar Pradesh,257.0,222.0,203.0,195.0,186.0,185.0,154.0,151.0,129.0,...,13.0,12.0,12.0,10.0,9.0,7.0,2.0,1.0,3749.0,NaN
proportion%,Uttar Pradesh,6.855161,5.921579,5.414777,5.201387,4.961323,4.934649,4.107762,4.027741,3.440918,...,0.346759,0.320085,0.320085,0.266738,0.240064,0.186716,0.053348,0.026674,100.0,NaN
count,Maharashtra,602.0,537.0,446.0,408.0,479.0,473.0,479.0,242.0,311.0,...,21.0,34.0,28.0,12.0,40.0,7.0,23.0,NaN,9049.0,NaN
proportion%,Maharashtra,6.652669,5.934357,4.928721,4.508786,5.293403,5.227097,5.293403,2.674329,3.436844,...,0.23207,0.375732,0.309426,0.132611,0.442038,0.077357,0.254172,NaN,100.0,NaN
count,Jharkhand,113.0,105.0,85.0,57.0,93.0,78.0,88.0,53.0,57.0,...,6.0,16.0,5.0,6.0,11.0,3.0,6.0,NaN,1659.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
proportion%,Dadra and Nagar Haveli and Daman and Diu,NaN,5.882353,11.764706,NaN,11.764706,5.882353,11.764706,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NaN
count,Dadra and Nagar Haveli,2.0,3.0,NaN,1.0,2.0,1.0,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,NaN
proportion%,Dadra and Nagar Haveli,10.0,15.0,NaN,5.0,10.0,5.0,NaN,NaN,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NaN
count,Mizoram,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN
